**Proyecto en grupo (2 miembros) - opción A:** El proyecto en grupo consiste en entrenar y evaluar varios modelos de clasificación supervisada capaz de clasificar una noticia según la taxonomía siguiente: 

- 1- Mundo, 2- Economía, 3- Política y Conflictos, 4- Ciencias y Tecnología, 5- Catástrofes y Accidentes, 6- Cultura y Artes, 7- Deporte, 8- Ecología y Planeta, 9- Crimen, delitos y Justicia, 10- Salud

Hito Unidad 1 (29 de septiembre): Datasets de entrenamiento y test + primer modelo baseline

Hito Unidad 2 (27 de octubre): Implementación y experimentos de varios modelos de clasificación

## (mejorar detalle del desarrollo y explicacion de codigos)

# Generando el dataset para el proyecto de clasificacion de noticias

Utilizamos los archivos csv proporcionados por el profesor para crear nuestro dataset

In [43]:
import pandas as pd
from os import listdir
from os.path import join
from tqdm import tqdm
from my_funcs import clean_and_tag
import re
from pandasql import sqldf
import numpy as np

In [33]:
pd.read_csv('./noticias/chile_2021-01.csv').head(3)

,id,country,media_outlet,url,title,text,date,year,id_journalist
0,3816243,chile,elmostrador,https://www.elmostrador.cl/dia/2021/01/01/ley-...,Ley de Migración: diputada Núñez (RN) formula ...,La diputada y vicepresidenta de Renovación Nac...,2021-01-01,2021.0,NaN
1,3816387,chile,elmostrador,https://www.elmostrador.cl/destacado/2021/01/0...,2021 arranca con 3.588 nuevos contagios y 52 f...,"Según el reporte de hoy, el Ministerio de Salu...",2021-01-01,2021.0,NaN
2,3816512,chile,elmostrador,https://www.elmostrador.cl/dia/2021/01/01/829-...,829 personas fueron detenidas durante noche de...,Carabineros reportó que hasta las 06:00 am. de...,2021-01-01,2021.0,NaN


 ya que estas noticias no se encuentran etiquetadas es necesario hacer magia asignarles una categoria que nos permita entrenar y probar nuestros modelos de clasificacion
 
 ### Categorias a utilizar
 
* 1- Mundo
* 2- Economía
* 3- Política y Conflictos
* 4- Ciencias y Tecnología,
* 5- Catástrofes y Accidentes,
* 6- Cultura y Artes, 
* 7- Deporte,
* 8- Ecología y Planeta,
* 9- Crimen, delitos y Justicia, 
* 10- Salud

### Adoptamos dos estrategias para realizar esta tarea:

1. asignarle una categoria a partir de la informacion de la url
2. utilizar LDA para encontrar topicos que correspondan a alguna categortia

## Etiquetado a partir de la informacion contenida en la url
... logica detras de esta decision

In [34]:
unique_url_tags = set()
folder = './noticias'
dfs = []
for archivo in tqdm(listdir(folder)):
    df = pd.read_csv(join(folder, archivo))
    corpus = clean_and_tag(df)
    dfs.append(corpus)
    for tag in corpus.tag.unique():
        unique_url_tags.add(tag)

100%|█████████████████████████████████████████████| 8/8 [00:03<00:00,  2.09it/s]


In [35]:
print(len(unique_url_tags))
list(unique_url_tags)[:10]

83


['turismo',
 'regiones',
 'dia',
 'columnas',
 'mineria',
 'especiales',
 'nacional',
 'plebiscito-chile-elige',
 'conversacioneslt',
 'Internacional']

encontramos 83 tags distintos. revisamos esta lista y asignamos manualmente una categoria a los url_tags que pertenecieran a alguna de las 10 categorias objetivo (archivo ```categorias - hojav2.csv```)

In [36]:
dm = pd.read_csv('./categorias - hojav2.csv')
dm.rename(columns={'Unnamed: 0':'tag'}, inplace=True)
vals = dm.iloc[:, 1:]
labels = vals[vals.max(axis=1)>0].idxmax(axis=1)
print(len(labels))
labels.head(5)

40


1       8-Ecología_y_Planeta
2          6-Cultura_y_Artes
3    4-Ciencias_y_Tecnología
4    3-Política_y_Conflictos
8    4-Ciencias_y_Tecnología
dtype: object

mapeo de url a categorias con un dict

In [37]:
cats = pd.DataFrame(labels)
cats = cats.join(dm['tag'])
cats.rename(columns={0:'cat'}, inplace=True)
cats_dict = dict(zip(cats['tag'], cats['cat']))
list(cats_dict.items())[:5]

[('animal', '8-Ecología_y_Planeta'),
 ('artes', '6-Cultura_y_Artes'),
 ('Autos', '4-Ciencias_y_Tecnología'),
 ('braga', '3-Política_y_Conflictos'),
 ('ciencia-tecnologia', '4-Ciencias_y_Tecnología')]

agregar la columna ```category``` al dataframe rellenando con Nan. tambien creamos una columna content que tendra el titulo mas el cuerpo de la noticia debido a ...

In [41]:
df = pd.concat(dfs)
df.drop_duplicates(subset='url', keep='first', inplace=True)
df['category'] = df.apply(lambda row : cats_dict.get(row.tag, np.nan), axis=1)
#df['content'] = df['title'] + df['text']
df['content'] = df[['title', 'text']].agg('. '.join, axis=1)
df.head(3)

,id,country,media_outlet,url,title,text,date,tag,category,content
0,12872,chile,horas24,https://www.24horas.cl/nacional/gobierno-da-luz-verde-a-temporada-de-piscinas-y-anuncio-reapertura-de-tupahue-4554782,Gobierno da luz verde a temporada de piscinas y anunció reapertura de Tupahue,"Con un acto en la piscina Tupahue, ubicada en el Parque Metropolitano, el Gobierno dio luz verde a la reapertura del...",2020-12-01,nacional,NaN,"Gobierno da luz verde a temporada de piscinas y anunció reapertura de Tupahue. Con un acto en la piscina Tupahue, ub..."
3,12929,chile,horas24,https://www.24horas.cl/nacional/subsecretaria-bown-la-defensoria-de-la-ninez-es-un-organo-autonomo-y-nos-gustaria-qu...,"Subsecretaria Bown: ""La Defensoría de la Niñez es un órgano autónomo y nos gustaría que se notara un poco más""","""Entendemos que la Defensoría es un órgano autónomo y nos gustaría que se notara un poco más"". Así comentó la subsec...",2020-12-01,nacional,NaN,"Subsecretaria Bown: ""La Defensoría de la Niñez es un órgano autónomo y nos gustaría que se notara un poco más"". ""Ent..."
4,12942,chile,horas24,https://www.24horas.cl/nacional/contralor-jorge-bermudez-asume-como-presidente-de-la-junta-de-auditores-de-la-onu-45...,Contralor Jorge Bermúdez asume como presidente de la Junta de Auditores de la ONU,El contralor Jorge Bermúdez asumió este 1 de diciembre la presidencia de la Junta de Auditores de Naciones Unidas re...,2020-12-01,nacional,NaN,Contralor Jorge Bermúdez asume como presidente de la Junta de Auditores de la ONU. El contralor Jorge Bermúdez asumi...


guardamos 30000 de las noticias cuyo tag era muy general o no nos servia para asignarle una categoria, mas adelante aplicaremos lda sobre estas noticias para etiquetar mas

In [42]:
pd.set_option('display.max_colwidth', 120)
df = df[['id','url','title', 'content', 'category']]
unravel = df[df['category'].isna()]
(unravel.sample(n=30000, random_state=2)).to_csv('30k_sin_cat.csv')
df = df[df['category'].notna()]
final_df_link = df.drop(columns=['url'])
final_df_link.to_csv('final_df_link.csv')
final_df_link.head(3)

,id,title,content,category
14,13115,"Vacunación en Chile empezaría primer trimestre del 2021 e incluiría a 15,2 millones de personas","Vacunación en Chile empezaría primer trimestre del 2021 e incluiría a 15,2 millones de personas. Este martes, el Pr...",10-Salud
18,13183,"Sernapesca: ""Viabilidad del virus de infectar es muy baja"" tras hallazgo de COVID-19 en paquete chileno en China","Sernapesca: ""Viabilidad del virus de infectar es muy baja"" tras hallazgo de COVID-19 en paquete chileno en China. La...",10-Salud
125,614680,"Presidente de México asegura que la pandemia ""no nos ha rebasado""","Presidente de México asegura que la pandemia ""no nos ha rebasado"". El presidente de México, Andrés Manuel López Obra...",1-Mundo


In [ ]:
# final_df.to_csv('cats_link.csv')

podemos ver que con esta tecnica logramos etiquetar una cantidad considerable de notocias en alguas categorias pero otras no tanto (falta la 5) ademas lo de los segos covid y todas esas manos...

In [ ]:
q="""SELECT category, count(*) FROM final_df GROUP BY category ORDER BY count(*) DESC;"""
result=sqldf(q)
result

## utilizando LDAb
![alt text](data/LDAB.jpg "Title")

a las 30k noticias que guaradmos antes le aplicamos lda con 20 topicos a lo maldiot
por que 20? porque si

In [6]:
import re
from pprint import pprint
import nltk;
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from nltk.corpus import stopwords
import string
# spacy for lemmatization
import spacy
# Plotting tools
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
df = pd.read_csv('./30k_sin_cat.csv')
df.drop(columns=['Unnamed: 0'], inplace=True)
df.head(3)

,id,url,title,content,category
0,7329456,https://www.meganoticias.cl/tendencias/320596-...,Adamari López y su lucha contra el sobrepeso: ...,Adamari López y su lucha contra el sobrepeso: ...,NaN
1,181069,https://www.biobiochile.cl/noticias/nacional/r...,Encuentran cuerpo de médico en río San José de...,Encuentran cuerpo de médico en río San José de...,NaN
2,4829009,https://www.latercera.com/nacional/noticia/int...,Intendente de la RM por paro de camioneros: “I...,Intendente de la RM por paro de camioneros: “I...,NaN


In [8]:
def sent_to_words(sentences):
    for sentence in tqdm(sentences):
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

In [9]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in tqdm(texts)]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in tqdm(texts)]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in tqdm(texts):
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [10]:
stop_words = stopwords.words('spanish')
# Convert to list
data = df.content.values.tolist()
# Remove new line characters
data = [re.sub(r'\s+', ' ', sent) for sent in tqdm(data)]

100%|███████████████████████████████████| 30000/30000 [00:04<00:00, 6502.09it/s]


In [11]:
data_words = list(sent_to_words(data))

100%|███████████████████████████████████| 30000/30000 [00:25<00:00, 1161.06it/s]


In [12]:
%%time

# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

CPU times: user 59 s, sys: 315 ms, total: 59.3 s
Wall time: 59.5 s


In [13]:
nlp = spacy.load('es_core_news_md', disable=['parser', 'ner'])
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams,
                                allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

100%|█████████████████████████████████████| 30000/30000 [09:13<00:00, 54.23it/s]


In [14]:
id2word = corpora.Dictionary(data_lemmatized)
texts = data_lemmatized
corpus = [id2word.doc2bow(text) for text in tqdm(texts)]

100%|███████████████████████████████████| 30000/30000 [00:03<00:00, 9440.30it/s]


### 4. Entrenamiento del modelo de tópico con LDA

- Entrenaremos un primer modelo de tópicos buscando un modelo con 20 tópicos. 

In [20]:
model_file = "./models/model_lda_20_topics.mm"
num_topics = 20
#lda_model = gensim.models.ldamodel.LdaModel.load("./models/model_lda_20_topics.mm", )

In [23]:
try:
    lda_model = gensim.models.ldamodel.LdaModel.load("./models/model_lda_20_topics.mm")
    id2wordxd = corpora.Dictionary.load(model_file + ".id2word")  
except FileNotFoundError:
    id2word = corpora.Dictionary(data_lemmatized) 
    lda_model = gensim.models.ldamulticore.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topics, 
                                           random_state=num_topics,
                                           chunksize=100,
                                           passes=10,
                                           per_word_topics=True,
                                                   workers=4)
    lda_model.save(model_file)
texts = data_lemmatized
#corpus = [id2word.doc2bow(text) for text in tqdm(texts)]
corpusxd = [id2wordxd.doc2bow(text) for text in tqdm(texts)]

100%|███████████████████████████████████| 30000/30000 [00:03<00:00, 9246.96it/s]


In [24]:
def format_topics_documents(ldamodel, corpus, texts):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(tqdm(ldamodel[corpus])):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

In [25]:
df_topic_sents_keywords = format_topics_documents(lda_model, corpusxd, texts)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'text_list']
df_dominant_topic.head(20)

100%|█████████████████████████████████████| 30000/30000 [08:26<00:00, 59.27it/s]


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,text_list
0,0,6.0,0.6294,"post_on, instagrama_post, view_this, post_shar...","[adamari_lopez, luchar, sobrepeso, montana_rus..."
1,1,12.0,0.6791,"hecho, vehiculo, lugar, hombre, encontrar, per...","[encontrar, cuerpo, medico, llevar, mes, desap..."
2,2,15.0,0.3296,"hacer, decir, ver, creer, querer, gente, mas, ...","[intendente, paro, camionero, interrumpir, lib..."
3,3,8.0,0.3821,"servicio, producto, persona, encontrar, usuari...","[dentro, dar, conocer, experiencia, usuario, n..."
4,4,7.0,0.4914,"chileno, ano, nuevo, primero, argentino, obra,...","[homenajear, justicia, espada_acuna, primero, ..."
5,5,6.0,0.8272,"post_on, instagrama_post, view_this, post_shar...","[rafael_araneda, marcela_vacarezza, presentar,..."
6,6,4.0,0.6721,"vacuna, salud, persona, dosis, primero, virus,...","[escapista, variante, africano, podrio, dismin..."
7,7,16.0,0.3591,"carabinero, persona, policial, general, violen...","[estruendo, alertar, comuna, rm, descartar, ex..."
8,8,4.0,0.7228,"vacuna, salud, persona, dosis, primero, virus,...","[detalle, proceso, vacunacion, deposito, nivel..."
9,9,15.0,0.7055,"hacer, decir, ver, creer, querer, gente, mas, ...","[padre, separar, parte, llegar, terraza, cafe,..."


In [26]:
df_topics = pd.concat([df.drop(columns=['category']), df_dominant_topic[['Dominant_Topic', 'Keywords', 'text_list']]], axis=1)

In [27]:
df_topics['Dominant_Topic'] = df_topics['Dominant_Topic'].astype(int)
df_topics.head(3)

,id,url,title,content,Dominant_Topic,Keywords,text_list
0,7329456,https://www.meganoticias.cl/tendencias/320596-...,Adamari López y su lucha contra el sobrepeso: ...,Adamari López y su lucha contra el sobrepeso: ...,6,"post_on, instagrama_post, view_this, post_shar...","[adamari_lopez, luchar, sobrepeso, montana_rus..."
1,181069,https://www.biobiochile.cl/noticias/nacional/r...,Encuentran cuerpo de médico en río San José de...,Encuentran cuerpo de médico en río San José de...,12,"hecho, vehiculo, lugar, hombre, encontrar, per...","[encontrar, cuerpo, medico, llevar, mes, desap..."
2,4829009,https://www.latercera.com/nacional/noticia/int...,Intendente de la RM por paro de camioneros: “I...,Intendente de la RM por paro de camioneros: “I...,15,"hacer, decir, ver, creer, querer, gente, mas, ...","[intendente, paro, camionero, interrumpir, lib..."


In [28]:
from my_funcs import get_df_keywords
get_df_keywords(lda_model, num_topics)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,agua,region,caso,candidato,vacuna,proyecto,post_on,chileno,servicio,colegio,persona,presidente,hecho,ano,mujer,hacer,carabinero,alcalde,plebiscito,caso
1,ambiental,zona,publico,partido,salud,gobierno,instagrama_post,ano,producto,ano,paso,ministro,vehiculo,millón,menor,decir,persona,sector,voto,nuevo
2,pueblo,hora,delito,primario,persona,ministro,view_this,nuevo,persona,educacion,comuna,diputado,lugar,nuevo,genero,ver,policial,incendio,votar,contagio
3,comunidad,biobio,fiscal,independiente,dosis,ley,post_shared,primero,encontrar,profesor,medida,politico,hombre,mas,violencia,creer,general,trabajador,proceso,total
4,pueblos_originario,rio,fiscalia,candidatura,primero,diputado,shared_by,argentino,usuario,estudiante,fase,ex,encontrar,mayor,adolescente,querer,violencia,vecino,presidente,cifra
5,territorio,nacional,ministerio,alcalde,virus,presidente,ano,obra,sistema,clase,salud,cargo,persona,estudio,nino,gente,detenido,municipio,politico,salud
6,tierra,eclipse,hecho,acuerdo,paciente,hacer,by,mundo,contar,tambien,cuarentena,parlamentario,detenido,tambien,servicio,mas,hecho,común,electoral,persona
7,derecho,sismo,tribunal,ir,decir,parlamentario,hijo,serie,realizar,salud,plan,senador,ano,desarrollo,edad,persona,personal,trabajar,octubre,nivel
8,mapuche,total,ano,constituyente,enfermedad,comision,hacer,temporada,empresa,alumno,semana,jefe,joven,empresa,talcahuano,dar,institucion,trabajo,apruebo,aumento
9,isla,región,imputado,politico,coronavirus,retiro,vida,historia,hora,nino,permiso,decir,sujeto,nivel,familia,tambien,publico,bombero,votacion,región


analisis alan tigua igual que con las url nos entrega el siguiente mapeo

In [29]:
cats_dict = {5 : '9-Crimen_delitos_y_Justicia',
             7 : '9-Crimen_delitos_y_Justicia',
             9 : '3-Política_y_Conflictos',
             10 : '3-Política_y_Conflictos',
             13 : '2-Economía',
             15 : '4-Ciencias_y_Tecnología',
             16 : '1-Mundo',
             18 : '5-Catástrofes_y_Accidentes',}

# el resto aun sin categoria
for i in range(0, num_topics):
    if i not in cats_dict.keys():
        cats_dict[i] = np.nan
        
cats_dict
df_topics['category'] = df_topics.apply(lambda row : cats_dict[row.Dominant_Topic], axis=1)
df_topics.head(3)

,id,url,title,content,Dominant_Topic,Keywords,text_list,category
0,7329456,https://www.meganoticias.cl/tendencias/320596-...,Adamari López y su lucha contra el sobrepeso: ...,Adamari López y su lucha contra el sobrepeso: ...,6,"post_on, instagrama_post, view_this, post_shar...","[adamari_lopez, luchar, sobrepeso, montana_rus...",NaN
1,181069,https://www.biobiochile.cl/noticias/nacional/r...,Encuentran cuerpo de médico en río San José de...,Encuentran cuerpo de médico en río San José de...,12,"hecho, vehiculo, lugar, hombre, encontrar, per...","[encontrar, cuerpo, medico, llevar, mes, desap...",NaN
2,4829009,https://www.latercera.com/nacional/noticia/int...,Intendente de la RM por paro de camioneros: “I...,Intendente de la RM por paro de camioneros: “I...,15,"hacer, decir, ver, creer, querer, gente, mas, ...","[intendente, paro, camionero, interrumpir, lib...",4-Ciencias_y_Tecnología


In [40]:
for i in range(num_topics):
    # agregar solo los category == nan
    if not isinstance(cats_dict[i], str) and np.isnan(cats_dict[i]):
        sub_df = df_topics[df_topics['Dominant_Topic'] == i]
        sub_df = sub_df[['id', 'title', 'content', 'Dominant_Topic', 'text_list']]
        sub_df.to_csv(f'./dominant_topics/dominant_topic{i}.csv')

In [41]:
final_df_topics = df_topics[['id','url','title', 'content', 'category']]
final_df_topics = final_df_topics[final_df_topics['category'].notna()]
final_df_topics.drop(columns=['url'], inplace=True)
final_df_topics.to_csv('final_df_topic.csv')
final_df_topics.head(3)

,id,title,content,category
2,4829009,Intendente de la RM por paro de camioneros: “I...,Intendente de la RM por paro de camioneros: “I...,4-Ciencias_y_Tecnología
4,3808041,"Google homenajea a Justicia Espada Acuña, la p...","Google homenajea a Justicia Espada Acuña, la p...",9-Crimen_delitos_y_Justicia
7,13387486,Estruendo alerta a varias comunas de la RM: Bo...,Estruendo alerta a varias comunas de la RM: Bo...,1-Mundo


In [45]:
final_df_link = pd.read_csv('final_df_link.csv').drop(columns=['Unnamed: 0'])
final_df_link.head(3)

,id,title,content,category
0,13115,Vacunación en Chile empezaría primer trimestre...,Vacunación en Chile empezaría primer trimestre...,10-Salud
1,13183,"Sernapesca: ""Viabilidad del virus de infectar ...","Sernapesca: ""Viabilidad del virus de infectar ...",10-Salud
2,614680,"Presidente de México asegura que la pandemia ""...","Presidente de México asegura que la pandemia ""...",1-Mundo


In [46]:
final_df = pd.concat([final_df_link, final_df_topics]).reset_index().drop(columns=['index'])

In [48]:
q="""SELECT category, count(*) FROM final_df_topics GROUP BY category ORDER BY count(*) DESC;"""
result=sqldf(q)
result

,category,count(*)
0,3-Política_y_Conflictos,3408
1,9-Crimen_delitos_y_Justicia,3028
2,4-Ciencias_y_Tecnología,2181
3,2-Economía,1427
4,1-Mundo,1351
5,5-Catástrofes_y_Accidentes,1160


In [49]:
q="""SELECT category, count(*) FROM final_df_link GROUP BY category ORDER BY count(*) DESC;"""
result=sqldf(q)
result

,category,count(*)
0,1-Mundo,8609
1,7-Deporte,6708
2,4-Ciencias_y_Tecnología,4225
3,2-Economía,2636
4,3-Política_y_Conflictos,1923
5,10-Salud,333
6,6-Cultura_y_Artes,185
7,8-Ecología_y_Planeta,135
8,9-Crimen_delitos_y_Justicia,130


In [50]:
q="""SELECT category, count(*) FROM final_df GROUP BY category ORDER BY count(*) DESC;"""
result=sqldf(q)
result

,category,count(*)
0,1-Mundo,9960
1,7-Deporte,6708
2,4-Ciencias_y_Tecnología,6406
3,3-Política_y_Conflictos,5331
4,2-Economía,4063
5,9-Crimen_delitos_y_Justicia,3158
6,5-Catástrofes_y_Accidentes,1160
7,10-Salud,333
8,6-Cultura_y_Artes,185
9,8-Ecología_y_Planeta,135


## clasificacion de subtopicos en el otro cuadernillo

# Primeros entrenamiento de un modelo

In [ ]:
final_df = final_df.groupby('category').sample(n=130, random_state=2)
q="""SELECT category, count(*) FROM final_df GROUP BY category ORDER BY count(*) DESC;"""
result=sqldf(q)
result

In [ ]:
final_df.reset_index(drop=True, inplace=True)
final_df

In [ ]:
train_size = 0.70
train_idxs = final_df.groupby('category').sample(frac=train_size, random_state=2).index
train_df = final_df.iloc[train_idxs]

In [ ]:
test_idxs = pd.concat([final_df, train_df]).drop_duplicates(keep=False).index
test_df = final_df.iloc[test_idxs]
q="""SELECT category, count(*) FROM test_df GROUP BY category ORDER BY count(*) DESC;"""
result=sqldf(q)
result

In [ ]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
ylabels = final_df['category']

lb.fit(ylabels.unique())
true_labels = [lb.inverse_transform([i])[0] for i in range(ylabels.nunique())]
ylabels_train_target = lb.fit_transform(ylabels[train_idxs])
ylabels_test_target = lb.fit_transform(ylabels[test_idxs])

for label in true_labels:
    print(f" {label:<29} :  {lb.transform([label])[0]}")

In [ ]:
# Word tokenization
from spacy.lang.es import Spanish

# Load English tokenizer, tagger, parser, NER and word vectors
nlp = Spanish()

text = final_df.sample(n=1)['text'].values[0][:100]
#  "nlp" Object is used to create documents with linguistic annotations.
my_doc = nlp(text)

# Create list of word tokens
token_list = []
for token in my_doc:
    token_list.append(token.text)
print(token_list)

In [ ]:
#Stop words
#importing stop words from English language.
from spacy.lang.es.stop_words import STOP_WORDS

#Implementation of stop words:
filtered_sent=[]

#  "nlp" Object is used to create documents with linguistic annotations.
doc = nlp(text)

for word in doc:
    if word.is_stop==False:
        filtered_sent.append(word)
print("Filtered Sentence:",filtered_sent)

In [ ]:
# !python -m spacy download es_core_news_md

In [ ]:


nlp = spacy.load('es_core_news_md', disable=['parser','ner','textcat','...'] )
# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
nlp = spacy.load("es_core_news_md")
stop_words = STOP_WORDS

In [ ]:
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('spanish')

# Creating our tokenizer function
def spacy_tokenizer(sentence, stem=False):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = nlp(sentence)
    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() for word in mytokens]
    
    # Removing stop words, ver lo de puntuuacion con  token.tag_
    mytokens = [ word for word in mytokens 
                          if word not in stop_words 
                          and word not in punctuations ]
    if stem:
        mytokens = [ stemmer.stem(word) for word in mytokens ]
    # return preprocessed list of tokens
    return " ".join(mytokens)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import plot_confusion_matrix, classification_report
from functools import partial
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression

In [ ]:
%%time
vectorizer = TfidfVectorizer(preprocessor=partial(spacy_tokenizer, stem=False))
XD = vectorizer.fit_transform(final_df['text'].values)
feature_names = vectorizer.get_feature_names()

In [ ]:
X_train = XD[train_idxs]
X_test = XD[test_idxs]

In [ ]:
%%time
lgr = LogisticRegression(random_state=42, multi_class='ovr', class_weight=None)
lgr.fit(X_train, ylabels_train_target)

 ```
 1-Mundo      :  0
 10-Salud     :  1
 2-Economía   :  2
 3-Política_y_Conflictos :  3
 4-Ciencias_y_Tecnología :  4
 6-Cultura_y_Artes :  5
 7-Deporte    :  6
 8-Ecología_y_Planeta :  7
 9-Crimen_delitos_y_Justicia :  8
 ```

In [ ]:
def predict_and_compare(df, vectorized_df, idxs=(0, 6)):
    predictions = lgr.predict(vectorized_df[idxs[0]:idxs[1]])
    predict_probs = lgr.predict_proba(vectorized_df[idxs[0]:idxs[1]])
    for i, (pred, probs) in enumerate(zip(predictions, predict_probs)):
        print('\n', df.iloc[i+idxs[0]].url)
        print()
        print(df.iloc[i+idxs[0]].text[:200], '...')
        print()
        print(f'predicted ---{lb.inverse_transform([pred])[0]}---', end=' ')
        print(f'prob: {probs[pred]:.3f}')
        print('*'*60)

In [ ]:
# probar con los que no tienen etiquta real
unravel = unravel.sample(n=100)
unravel.reset_index(inplace=True)
owo = vectorizer.transform(unravel['text'].values)

In [ ]:
predict_and_compare(unravel, owo, idxs=(0, 5))

In [ ]:
def results(ylb, text_clf, X_test):
    predicted = text_clf.predict(X_test)
    clf_name = type(text_clf).__name__
    print(f"Resultados clasificación\n{clf_name}\n\n")
    print(classification_report(ylb, predicted, target_names=true_labels))
    fig, ax = plt.subplots(figsize=(7, 7), tight_layout=True)
    plot_confusion_matrix(text_clf, X_test, ylb,
                          display_labels=true_labels,
                          ax=ax,
                          cmap='Blues')
    ax.set_title(f"Matriz de Confusión\n{clf_name}")
    plt.xticks(rotation=60)
    plt.show()

In [ ]:
results(ylabels_test_target, lgr, X_test)

In [ ]:
text = final_df.sample(n=1)['text'].values[0][:100]
print(text)
spacy_tokenizer(text)

In [ ]:
doc = nlp(text[:20])
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
           token.shape_, token.is_alpha, token.is_stop)